In this notebook we suggest an approach to estimate future water levels of multiple waterbodies in northern and central Italy. Nine different waterbodies of 4 distinct types (acquifers, water springs, rivers & lakes) are studies, each with different target variables. Instead of aiming for the lowest possible error black box models, we aim to generate interpretable models that do not only give usable estimations, but also allow domain experts to interact with the models. External weather data is collected surrounding the waterbody locations, and we use custom dimension reduction techniques to engineer easy-to-interpret features. Those features are then used as inputs for Gradient Boosted Tree (GBM) models, which fit the natural processes of water bodies (@thomas: referentie). 

We aim for this notebook to be concise and to the point and will therefor not extensively analyse each of the 20 target measurements or all of the input parameters, but leave the reader with documented code to further dive into points of his or her interest.

# Executive summary

# Data gathering, cleaning and description 

Waterbodies are relatively simple to explain systems: the water level in the future depends on the current water level + the water that comes in - everything that goes out. What comes in and goes out is largely dependent on the weather: rain and evaporation interacting with the way the waterbody works. A river for instance will quickly collect rainwater, while a deeply buried aquifer might take a number of days. The usage of weather predictions are explicitely prohibited for this competition, so instead we aim to get a more detailed view of the current weather by collecting weather data from NASA's POWER system (https://power.larc.nasa.gov/) as external dataset. Based on the given names of the waterbodies, 57 locations of interest are determined, for each which 16 weather metrics (rain, humidity, temperature and wind speeds) are collected in a daily timeframe going back to 2000. We combine this external dataset with the given datasets and manually clean up a number inconsistencies in the data, the details of which are commented in the code. For the features, missing values are filled with a quadratic interpolation, the missing values of the target variables are left empty so we don't train models on guessed measurements, only on actual data. Since water levels are gamma-like distributed (@thomas reference) and can see significant changes due to weather and human interventions, we only clean the obviously faulty data such as 0-values, but remove no further outliers.

# Exploratory data analysis

- show why we np.log the target variables


# Methods

## Feature engineering

Feature engineering is done by dimension reduction of the weather data. Per data type (i.e. ws10m, the wind speed at 10m altitude, or rainfall), we multiply the data points D with a location weight matrix W and take the mean, where W is optimized. This approach offers several advantages over other dimension reduction techniques:
- The dimension reduction is significant, for example the 16 NASA POWER features over 10 locations (160 features) are reduced to 16 features
- The resulting features are still easy to interpret, as they directly represent weather conditions, such as "rain" or "wind speed"
- By hyperoptimizing W, we get an optimal location vector that directly represents how important a certain location is for a target variable 

We do this for both the NASA POWER data and the internal dataset that is given, resulting in the following features:
- 16 NASA POWER weather features (wind speed, temperature, etc.)
- target variables (# features is variable per dataset)
- Internal data set features (# features is variable per dataset)
- Several date related features: month (1-12, categorical), week (1-52, continuous) & day_of_week (1-7, categorical). The continuous week variable helps the model with seasonality, while the other 2 categorical variables assist in detecting date based artifacts, such as the ones found in target_flow_rate_galleria_alta. Adding yearly data often hurts performance (referentie?)

Finally some historical data needs to be taken into account. We add 8 shifted rolling means for each of the previously mentioned features: rolling mean of period 2 shifted back 1, 3 & 5 days, rolling mean of period 5 shifted back 5, 10 & 15 days, and the rolling mean of period 20 shifted back 20 and 40 days. This gives the model plenty of data to work with without getting a huge number of features.

## Methods
Using the engineered features, we split the data into a train, validation and test chunck in a 70-10-20 fashion, and train a GBM model. The training data is used as input, the validation data for early stopping and optimization of the location weight matrix W, and the test data is used for validation and metric reporting. Since we only train model on data where the target variable is non-null, the exact split dates differ per dataset. Since every target variable 

For every target variable we generate 3 models: predicting 14, 28 and 56 days ahead. Targets are not predicted directly, but transformed into a logged percentual change, so that 
`target_14 = log(target(t-14) / target(t) + 1)`. Using multiple models is preferable over a single one with multiple targets, since predicting 14 days ahead allows using training data up to t-14 (where t is the current day), while predicting 56 days ahead only allows data up to t-56.

Finally, we show that optimizing W generally improves the model, and that a rolling model - where the model is retrained after a number of days - delivers better scores.

# Findings

# Discussion

- The current measurements are likely influenced by human intervention, i.e. the opening or closing of dams. We could not find a dataset which described these actions so we couldn't account for it. If actions are taken based on the predictions of a model that's trained on this kind of biased data, there is risk of a feedback loop where the model makes predictions as if an intervening action will be made.